# Ch5: Introduction To Modern Symmetric Key Ciphers
## 5.1 Permutation Box 
A Permutation Box is a transposition Cipher that takes in a permutation and reorders the bits or a block of data in as per the permutation. There can be 3 types of PBox's:

1. Straigt P-Box (is invertible)
2. Compression P-Box (is non-invertible)
3. Expansion P-Box (is non-invertible)

We define a class `PBox` and define the permutation mapping for all 3 types of P-Boxes using a python dictionary.

In [5]:
class PBox:
    def __init__(self, key: dict):
        self.key = key
        self.in_degree = len(key)
        self.out_degree = sum(len(value) if isinstance(value, list) else 1 for value in key.values())

    def __repr__(self) -> str:
        return 'PBox' + str(self.key)

    def permutate(self, sequence: list) -> list:
        result = [0] * self.out_degree
        for index, value in enumerate(sequence):
            if (index + 1) in self.key:
                indices = self.key.get(index + 1, [])
                indices = indices if isinstance(indices, list) else [indices]
                for i in indices:
                    result[i - 1] = value
        return result

    def is_invertible(self) -> bool:
        return self.in_degree == self.out_degree

    def invert(self):
        if self.is_invertible():
            result = {}
            for index, mapping in self.key.items():
                result[mapping] = index
            return PBox(result)

In [6]:
# Compression P-Box
compression_p_box = PBox({1: 1, 2: [], 3: 2})
print('In Degree:', compression_p_box.in_degree)
print('Out Degree:', compression_p_box.out_degree)
print(compression_p_box.permutate([10, 20, 30]))

In Degree: 3
Out Degree: 2
[10, 30]


In [7]:
# Compression boxes are non-invertible
print(compression_p_box.is_invertible())

False


In [8]:
# Expansion P Box
expansion_p_box = PBox({1: 1, 2: 2, 3: [3, 4]})
print(expansion_p_box)
print('In Degree:', expansion_p_box.in_degree)
print('Out Degree:', expansion_p_box.out_degree)
print(expansion_p_box.permutate([10, 20, 30]))

PBox{1: 1, 2: 2, 3: [3, 4]}
In Degree: 3
Out Degree: 4
[10, 20, 30, 30]


In [9]:
# Expansion P Boxes are non invertible
print(expansion_p_box.is_invertible())

False


In [11]:
# Straight P Boxes
p_box = PBox({1: 3, 2: 1, 3: 2})
print('In Degree:', p_box.in_degree)
print('Out Degree:', p_box.out_degree)
print(p_box.permutate([10, 20, 30]))

In Degree: 3
Out Degree: 3
[20, 30, 10]


In [13]:
# straight P boxes are invertible
print(p_box.is_invertible())
print('Inverse:', p_box.invert())

True
Inverse: PBox{3: 1, 1: 2, 2: 3}


In [12]:
# composition of pemutation and inverse leads in the identity permutations
print(p_box.invert().permutate(p_box.permutate([10, 20, 30])))

[10, 20, 30]


## 5.2 Feistel Cipher
The Feistel Cipher is a product cipher and uses many different coomponents, even non-invertible components in the cipher. The non-invertible part is called the __mixer__ and uses the _XOR_ function to encrypt and decrypt a number using a secret key (also a number).

In [14]:
class FiestelMixerCipher:
    def __init__(self, key: int):
        self.key = key

    def encrypt(self, number: int) -> int:
        return number ^ self.key

    def decrypt(self, number: int) -> int:
        return self.encrypt(number)

In [15]:
fiestel_mixer = FiestelMixerCipher(key=9)
ciphertext = fiestel_mixer.encrypt(10)
print(ciphertext)

3


In [16]:
print(fiestel_mixer.decrypt(ciphertext))

10


In the Feistel cipher we can do many consecutive rounds of encryption wherin each encryption has a non-invertible function and also a swap operation. Below we define a single Instance of the `Round` class which will later on be used to compose a Feistel Cipher. 

A Fiestel Cipher with __n__ rounds is called a _n round Fiestel Cipher_.

In [17]:
def int_to_bin(number: int, block_size=8) -> str:
    binary = bin(number)[2:]
    return '0' * (block_size - len(binary)) + binary


def char_2_num(letter: str) -> int:
    return ord(letter) - ord('a')


class Round:
    def __init__(self, key: int, func, block_size=8):
        # key = int_to_bin(key)
        self.key = key
        self.func = func
        self.block_size = block_size

    def encrypt(self, plaintext_number: int) -> int:
        l1, r1 = self.binary_fragments(plaintext_number, self.block_size // 2)
        f1 = self.func(r1, self.key)
        r2 = f1 ^ l1
        l2 = r1
        result = int_to_bin(l2, self.block_size // 2) + int_to_bin(r2, self.block_size // 2)
        return int(result, base=2)

    def decrypt(self, cipher_number: int) -> int:
        l2, r2 = self.binary_fragments(cipher_number, self.block_size // 2)
        f = self.func(r2, self.key)
        l1 = r2 ^ f
        r1 = l2
        result = int_to_bin(l1, self.block_size // 2) + int_to_bin(r1, self.block_size // 2)
        return int(result, base=2)

    @staticmethod
    def binary_fragments(number, block_size=4):
        binary = int_to_bin(number, 2 * block_size)
        l = binary[0: block_size]
        r = binary[block_size:]
        return int(l, base=2), int(r, base=2)

In [19]:
#We define a function that will be passed as an argument and the Round Cipher can be composed using the function
def polynomial_mod(p1: int, p2: int) -> int:
    return p1 % p2

In [20]:
round_cipher = Round(key=15, func=polynomial_mod, block_size=32)
letter = 'z'
plaintext_number = char_2_num(letter)
print('plaintext number:', plaintext_number)

plaintext number: 25


In [21]:
# We now encrypt our data using a single instance of the Round Cipher
ciphertext_number = round_cipher.encrypt(plaintext_number)
print('ciphertext number:', ciphertext_number)

ciphertext number: 1638410


In [22]:
# We decrypt the number using the cipher
decrypted = round_cipher.decrypt(ciphertext_number)
print('decrypted:', decrypted)

decrypted: 25


An implimentation of Fiestel Cipher with multiple Rounds

In [23]:
class FiestelCipher:
    def __init__(self, rounds: list):
        self.rounds = rounds

    def encrypt(self, plain_number: int) -> list:
        for round in self.rounds:
            plain_number = round.encrypt(plain_number)
        return plain_number

    def decrypt(self, cipher_number):
        for round in self.rounds[::-1]:
            cipher_number = round.decrypt(cipher_number)
        return cipher_number

In [24]:
# We create an instance of the Fiestel Cipher with 2 rounds
def polynomial_mod(p1: int, p2: int) -> int:
    return p1 % p2


fiestel_cipher = FiestelCipher([
    Round(key=3, func=polynomial_mod, block_size=32),
    Round(key=7, func=polynomial_mod, block_size=128)
])

In [25]:
# we encrypt our message using a 2-round fiestel cipher
message = 24
cipher_number = fiestel_cipher.encrypt(message)
print(cipher_number)

29014219670751100192948230


In [26]:
# We decrypt using our Cipher
decrypted = fiestel_cipher.decrypt(cipher_number)
print(decrypted)

24


## 5.3 Modern Stream Ciphers
Stream Ciphers work upon streams of characters/bits of the data and act on one letter/bit at a time rather than acting on a block of fixed size.

### 5.3.1 Synchronous Stream Ciphers
In a synchronous stream cipher, the key stream is independent of the plaintext or ciphertext stream. Some examples of a synchronous stream ciphers are:

1. Vignere Cipher
1. One Time Pad Cipher 

### 5.3.2 Asynchronous Stream Ciphers